# Preprocesamiento para Experimentos de Actividad Espontánea
 
Este Cuaderno es muy similar al Preprocesamiento para Evocados, pero sin promediar sobre múltiples estímulos. En lugar de eso se recomienda una inspección visual para obtener un intervalo interesante del Archivo Experimental.

In [1]:
include("SeparaActividadySaturados01.jl")
using SeparaActividadySaturados01
using JLD

La mayoría de los experimentos de actividad espontánea producen archivos de varios GB. Es imposible hacer el prerprocesamiento con 
memoría limitada. Es muy probable que haya que dividir el archivo en cachos.

In [2]:
abrestring="/home/karel/BRWFiles/140116s2cut.brw"
stringgeneral=replace(abrestring, ".brw", "")  

"/home/karel/BRWFiles/140116s2cut"

In [3]:
Datos=AbreyCheca(abrestring)

Dict{String,Any} with 8 entries:
  "minvolt"     => -4125.0
  "maxvolt"     => 4125.0
  "frecuencia"  => 7022.06
  "DatosCrudos" => UInt16[0x0fff 0x0fff … 0x0fff 0x0fff; 0x0805 0x07f7 … 0x07ed…
  "bitdepth"    => 0x0c
  "factor"      => 2.01416
  "numcuadros"  => 2106617
  "duracion"    => 300.0

In [5]:
fmemlibre=Sys.free_memory()/Sys.total_memory()

0.14881024684492145

In [11]:
# Si los datos andan en un arreglo de lista en lugar de cuadrado, los ponemos cuadrados
if size(Datos["DatosCrudos"])[1] != 4096
DatosCrudosArreglados=reshape(Datos["DatosCrudos"], (4096, Datos["numcuadros"]))
else
DatosCrudosArreglados=Datos["DatosCrudos"]
end;

# si la fraccion de memoria libre es inferior a 0.2, tenemos que cortar un cacho de los datos. Que tan chico el cacho depende... 
# toca probar
if fmemlibre <0.2
cacho=round(Int, Datos["numcuadros"]/4)
DatosCrudosArreglados=DatosCrudosArreglados[:,1:cacho]
    end;
# Liberar memoria
Datos["DatosCrudos"]=0

0

In [12]:
### parametetros globales.
freq=Datos["frecuencia"]/1000 #cuadros por milisegundo.
factor=Datos["factor"] #Factor de conversion de numeros enteros a microVolts
retrazo=round(Int, ceil(5*freq))
final=round(Int, ceil(60*freq))
latencia=round(Int, ceil(1.5*freq))


11

In [13]:
 DatosCentrados=FormaMatrizDatosCentrados(DatosCrudosArreglados, factor);
# libera memoria
DatosCrudosArreglados=0


0

In [15]:
size(DatosCentrados)

(64, 64, 526654)

In [16]:
cuadrosmax=size(DatosCentrados)[3]
tiempototalms=cuadrosmax/freq 
println("Tienes ", cuadrosmax, " cuadros de muestreo a ", freq, " cuadros por segundo. Esto corresponde a  ", tiempototalms, "ms." )

Tienes 526654 cuadros de muestreo a 7.022058198545425 cuadros por segundo. Esto corresponde a  74999.94803647356ms.


In [21]:
#los parametros son datos promediados, frecuencia, tiempo post estimulo en ms, umbral en microvolts, umbral de saturación en microvolts
PruebaRespuesta=BuscaCanalRespActPot(DatosCentrados,freq, 120,-120,-800)
# y luego los saturados
iniciobusqueda=retrazo
finbusqueda=retrazo+4*latencia
Saturados=BuscaSaturados(DatosCentrados,1200,iniciobusqueda,finbusqueda)
numerosaturados=length(Saturados)
println("Encontramos ", numerosaturados, " canales probablemente saturados.")
println("Encontramos ", length(PruebaRespuesta) " canales probablemente con actividad.")


Encontramos 5 canales probablemente saturados.


Set(Array{Int64,1}[[10, 46], [32, 1], [29, 34], [49, 46], [19, 36], [6, 9], [8, 32], [44, 41], [34, 53], [46, 14]  …  [30, 40], [54, 42], [4, 20], [48, 30], [40, 47], [14, 57], [35, 25], [17, 48], [13, 56], [44, 55]])